### Queries na base de dados Mongo


In [1]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [2]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [3]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [4]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [5]:
#df = pd.DataFrame.from_records(processos,index=['nro_processo','link_processo','nome_parte'])
dados = pd.DataFrame(list(col_processos.find({})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

(48204, 8)

In [6]:
# Processos por empresa
df_processos.groupby('cod_empresa')['nro_processo'].count()

cod_empresa
ALLIANZ         4315
AZUL            5682
EMS               91
GOL             4195
MAPFRE          5281
MLUIZA          1775
N/I               38
PORTOSEGURO    14436
SULAMERICA      3317
TAM             6239
TOKIOMARINE     2832
Name: nro_processo, dtype: int64

# Decisões

In [7]:
# Carrega base de decisões total
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])

In [8]:
df_decisoes.shape

(13850, 10)

In [20]:
col_decisoes.count_documents({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})

871

In [ ]:
decisao = col_decisoes.find_one({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})
decisao['decisao']

In [ ]:
# Decisões por Magistrado
df_qtdedecisoes = pd.DataFrame({"decisoes" : df_decisoes.groupby('magistrado').size()}).reset_index()
df_qtdedecisoes.sort_values(by="decisoes",ascending=False)